In [ ]:
# test
import pandas as pd

In [ ]:
pd.DataFrame({'Yes': [50, 21], 'No': [131, 2]})

In [ ]:
pd.DataFrame({'Bob': ['I liked it.', 'It was awful.'],
              'Sue': ['Pretty good.', 'Bland.']},
             index=['Product A', 'Product B'])

In [ ]:
pd.Series([30, 35, 40], index=['2015 Sales', '2016 Sales', '2017 Sales'], name='Product A')

In [ ]:
# wine_reviews = pd.read_csv("../input/wine-reviews/winemag-data-130k-v2.csv", index_col=0)
# wine_reviews = pd.read_csv("../input/wine-reviews/winemag-data-130k-v2.csv")
reviews = pd.read_csv("../input/wine-reviews/winemag-data-130k-v2.csv", index_col=0)

In [ ]:
reviews.country
reviews['country']
reviews['country'][0]
reviews.iloc[0]

In [ ]:
reviews.iloc[:, 0]
reviews.iloc[:3, 0]
reviews.iloc[1:3, 0]
reviews.iloc[[0, 1, 2], 0]
reviews.iloc[-5:]

In [ ]:
reviews.loc[0, 'country']
reviews.loc[:, ['taster_name', 'taster_twitter_handle', 'points']]


Choosing between loc and iloc¶
When choosing or transitioning between loc and iloc, there is one "gotcha" worth keeping in mind, which is that the two methods use slightly different indexing schemes.

iloc uses the Python stdlib indexing scheme, where the first element of the range is included and the last one excluded. So 0:10 will select entries 0,...,9. loc, meanwhile, indexes inclusively. So 0:10 will select entries 0,...,10.

Why the change? Remember that loc can index any stdlib type: strings, for example. If we have a DataFrame with index values Apples, ..., Potatoes, ..., and we want to select "all the alphabetical fruit choices between Apples and Potatoes", then it's a lot more convenient to index df.loc['Apples':'Potatoes'] than it is to index something like df.loc['Apples', 'Potatoet'] (t coming after s in the alphabet).

This is particularly confusing when the DataFrame index is a simple numerical list, e.g. 0,...,1000. In this case df.iloc[0:1000] will return 1000 entries, while df.loc[0:1000] return 1001 of them! To get 1000 elements using loc, you will need to go one lower and ask for df.loc[0:999].

Otherwise, the semantics of using loc are the same as those for iloc.

In [ ]:
reviews.set_index("title")
reviews.loc[(reviews.country == 'Italy') & (reviews.points >= 90)]
reviews.loc[reviews.country.isin(['Italy', 'France'])]
reviews.loc[reviews.price.notnull()]

In [ ]:
#adding data
reviews['critic'] = 'everyone'
reviews['index_backwards'] = range(len(reviews), 0, -1)


In [ ]:
reviews.points.describe()
reviews.taster_name.describe()
reviews.points.mean()
reviews.taster_name.unique()


In [ ]:
#  list of unique values and how often they occur in the dataset
reviews.taster_name.value_counts()
# or
reviews.groupby('points').points.count()


reviews.points.mean()

In [ ]:
# map and apply
review_points_mean = reviews.points.mean()
reviews.points.map(lambda p: p - review_points_mean)

def remean_points(row):
    row.points = row.points - row.points.mean()
    return row

reviews.apply(remean_points, axis='columns')#on each columns
reviews.apply(remean_points, axis='index')#on each rows

In [ ]:
bargain_idx = (reviews.points / reviews.price).idxmax()

In [ ]:
#  "tropical" or "fruity"?
# Create a Series descriptor_counts counting how many times each of these two words
# appears in the description column in the dataset. (For simplicity, let's
# ignore the capitalized versions of these words.)

n_trop = reviews.description.map(lambda desc: "tropical" in desc).sum()
n_fruity = reviews.description.map(lambda desc: "fruity" in desc).sum()
descriptor_counts = pd.Series([n_trop, n_fruity], index=['tropical', 'fruity'])

In [ ]:
# rating system ranging from 80 to 100 points is too hard to understand
# - we'd like to translate them into simple star ratings. A score of 95
# or higher counts as 3 stars, a score of at least 85 but less than 95 is
# 2 stars. Any other score is 1 star.
# Also, the Canadian Vintners Association bought a lot of ads on the site,
# so any wines from Canada should automatically get 3 stars, regardless of points.
def stars(row):
    if row.country == 'Canada':
        return 3
    elif row.points >= 95:
        return 3
    elif row.points >= 85:
        return 2
    else:
        return 1

star_ratings = reviews.apply(stars, axis='columns')

In [ ]:
reviews.groupby('points').price.min()
# return min price that is available for "each" point

In [ ]:
# What is the best wine I can buy for a given amount of money?(money from low to high)
best_rating_per_price = reviews.groupby('price')['points'].max().sort_index()

In [ ]:
# What are the minimum and maximum prices for each variety of wine?
reviews.groupby('variety').price.agg([min, max])

In [ ]:
# What are the most expensive wine varieties?
sorted_varieties = price_extremes.sort_values(by=['min', 'max'], ascending=False)

In [ ]:
# What combination of countries and varieties are most common?
country_variety_counts = reviews.groupby(['country', 'variety']).size().sort_values(ascending=False)
# solution:
# country  variety
# US       Pinot Noir                  9885
#          Cabernet Sauvignon          7315
#          Chardonnay                  6801
# France   Bordeaux-style Red Blend    4725
# Italy    Red Blend                   3624
#                                      ...
# Mexico   Cinsault                       1
#          Grenache                       1
#          Merlot                         1
#          Rosado                         1
# Uruguay  White Blend                    1
# Length: 1612, dtype: int64

In [ ]:
reviews.price.dtype
# dtype('float64')
reviews.points.astype('float64')
reviews[pd.isnull(reviews.country)]
reviews[pd.notnull(reviews.country)]
reviews.region_2.fillna("Unknown")
reviews.taster_twitter_handle.replace("@kerinokeefe", "@kerino")
n_missing_prices  = reviews.price.isnull().sum()

# finding most common regions
reviews.region_1.fillna('Unknown').value_counts().sort_values(ascending=False)

In [ ]:
reviews.rename(columns={'points': 'score'})
reviews.rename(index={0: 'firstEntry', 1: 'secondEntry'})
# also set_index()
# renaming main index axises:
reviews.rename_axis("wines", axis='rows').rename_axis("fields", axis='columns')

In [ ]:
# combines:
pd.concat([canadian_youtube, british_youtube])
# join:
left = canadian_youtube.set_index(['title', 'trending_date'])
right = british_youtube.set_index(['title', 'trending_date'])

left.join(right, lsuffix='_CAN', rsuffix='_UK')
# another example: both tables have a column "MeetID"
a=powerlifting_meets.set_index("MeetID")
b=powerlifting_competitors.set_index("MeetID")
powerlifting_combined = a.join(b)

In [ ]:
#region_1 and region_2  columns renamed to region and locale, respectively.
reviews.rename(columns=dict(region_1='region', region_2='locale'))
# Set the index name in the dataset to wines.
reindexed = reviews.rename_axis('wines', axis='rows')
